## ARYAN SONI BURMAN - 1761012

In [51]:
import nltk
# Library import for Tokenization of words
from nltk.tokenize import word_tokenize
# Library import for stopwords
from nltk.corpus import stopwords
# Library import for Lammatizer
from nltk.stem import WordNetLemmatizer

In [52]:
text = "movie is awesome I like watching it".lower()

In [53]:
#TEST PREPROCESSING
stop_words = set(stopwords.words("english"))
words = word_tokenize(text)
words_filtered=[]
for word in words:
    if word not in stop_words:
        words_filtered.append(word)

lemmatizer = WordNetLemmatizer() 
lemmatized_words = []
for word in words_filtered:
    lemmatized_words.append(lemmatizer.lemmatize(word))

In [54]:
pos_tagged_words = nltk.pos_tag(lemmatized_words)

In [55]:
pos_tagged_words

[('movie', 'NN'), ('awesome', 'VBP'), ('like', 'IN'), ('watching', 'VBG')]

In [56]:
'''TRAINGING CORPUS INCLUDES 3 POSITVE AND 2 NEGATIVE SENTENCE WITH 0 INDEX INDICATING CLASS OF SENTENCE AND 
    1 INDEX INDICATING THE ACTUAL SENTENCE'''

training_data = [
    ["P","movie is good direction was superb"],
    ["P","movie is good songs are just awesome like"],
    ["N","movie is not so good actor is superficial"],
    ["N","movie is not so good actress speaks dump"],
    ["P","movie is good real heroismlike"]
]

In [57]:
#CREATING POSITIVE AND NEGATIVE BAG OF SENTENCES
positive = []
negative = []
for value in training_data:
    if value[0] == "P":
        positive.append(value[1])
    elif value[0] == "N":
        negative.append(value[1])

In [58]:
#POSITIVE BAG OF WORDS
pos = []
for word in positive:
    token = nltk.word_tokenize(word)
    for w in token:
        pos.append(w.lower())

In [59]:
#NEGATIVE BAG OF WORDS
neg = []
for word in negative:
    token = nltk.word_tokenize(word)
    for w in token:
        neg.append(w.lower())

In [60]:
#REMOVE STOPWORDS FROM BOTH THE BAG OF WORDS(POSITIVE AND NEGATIVE)
pos_filtered = []
for word in pos:
    if word not in stop_words:
        pos_filtered.append(word)
neg_filtered = []
for word in neg:
    if word not in stop_words:
        neg_filtered.append(word)

In [61]:
pos_tagged_words

[('movie', 'NN'), ('awesome', 'VBP'), ('like', 'IN'), ('watching', 'VBG')]

In [62]:
#CREATING TABLE OF PROBABILITIES
total_words = len(pos_filtered)+len(neg_filtered)
table = []
for tup in pos_tagged_words:
    table.append([tup[0], pos_filtered.count(tup[0])/len(pos_filtered), 
                neg_filtered.count(tup[0])/len(neg_filtered),
                 (pos_filtered.count(tup[0])+neg_filtered.count(tup[0]))/total_words])

In [63]:
table

[['movie', 0.23076923076923078, 0.18181818181818182, 0.20833333333333334],
 ['awesome', 0.07692307692307693, 0.0, 0.041666666666666664],
 ['like', 0.07692307692307693, 0.0, 0.041666666666666664],
 ['watching', 0.0, 0.0, 0.0]]

In [64]:
#USING DATA FRAME TO DISPLAY THE VALUES
import pandas as pd
cols = ['Bag Of Words', 'Positive', 'Negative', 'P(X)']
dframe = pd.DataFrame(table, columns=cols)
dframe

,Bag Of Words,Positive,Negative,P(X)
0,movie,0.230769,0.181818,0.208333
1,awesome,0.076923,0.000000,0.041667
2,like,0.076923,0.000000,0.041667
3,watching,0.000000,0.000000,0.000000


In [65]:
#SUMMATION OF ALL POSITVE AND NEGATIVE PROBABILITIES
P_C_Positive = dframe['Positive'].sum(axis=0)
P_C_Negative = dframe['Negative'].sum(axis=0)

In [66]:
print(P_C_Positive, P_C_Negative)

0.38461538461538464 0.18181818181818182


## Bayes Theorum

In [67]:
#APPLYING BAYES THEORUM TO CALCULATE PROBABILITY OF EVERY WORD IN SENTENCE
sentiment_score = []
i=0
for word in dframe['Bag Of Words']:
    score = (dframe['Positive'][i]*P_C_Positive)/(dframe['Positive'][i]*P_C_Positive+dframe['Negative'][i]*P_C_Negative)
    sentiment_score.append([word, score])
    i = i+1

/home/aryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


In [68]:
sentiment_score

[['movie', 0.7286230429546368],
 ['awesome', 1.0],
 ['like', 1.0],
 ['watching', nan]]

In [69]:
#DISPLAYING WORDS AND THEIR SENTIMENT SCORE
dframe_score = pd.DataFrame(sentiment_score, columns=['Word', 'Score'])
dframe_score["Score"].fillna(0, inplace = True)
dframe_score

,Word,Score
0,movie,0.728623
1,awesome,1.000000
2,like,1.000000
3,watching,0.000000


In [70]:
#TEXT TO BE CLASSIFIED
print(text, ":",pos_tagged_words)

movie is awesome i like watching it : [('movie', 'NN'), ('awesome', 'VBP'), ('like', 'IN'), ('watching', 'VBG')]


In [71]:
#EXTRACTING THE NOUN AND ADJECTIVE FROM THE TEXT TO BE CLASSIFIED
used_words = []
for word in pos_tagged_words:
    if word[1] == 'NN' or word[1] == 'VBP' or word[1] == 'VB':
        used_words.append(word[0])
used_words

['movie', 'awesome']

In [72]:
#CALCULATING THE OVERALL SCORE FOR THE SENTENCE
sum = 0
for word in used_words:
    df = dframe_score[dframe_score['Word'] == word]
    sum += df['Score'].sum()

In [73]:
#CONVERTING TO PERCENTAGE
percent_prediction = (sum/2)*100

In [74]:
print(percent_prediction,"%")

86.43115214773184 %


In [75]:
y = percent_prediction
if y > 50:
    print('Sentence is Positive : Prediction Score = ',y, "percent")
else:
    print('Sentence is Negative : Prediction Score = ',y, "percent")

Sentence is Positive : Prediction Score =  86.43115214773184 percent
